In [53]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import pandas as pd
pd.options.mode.chained_assignment = None 

In [54]:
#loading the data
data = pd.read_csv("titanic.csv")

In [55]:
data.head()

,Survived,Pclass,Sex,Fare,Cabin,Embarked_imputed,Age_imputed,Family_Size
0,0,1,male,51.8625,rare,S,54.0,1
1,0,3,male,15.5000,missing,Q,25.0,1
2,0,2,male,41.5792,missing,C,25.0,4
3,0,3,male,14.4542,missing,C,26.0,2
4,0,3,female,10.5167,missing,S,22.0,1


In [56]:
#creating a subset of the data
data=data[["Sex","Cabin","Embarked_imputed","Survived"]]

In [57]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(data, # predictors
    data['Survived'],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)

# Weight-of-Evidence

In [58]:
#probablity of survived
prob = X_train.groupby("Sex")["Survived"].mean()
prob = pd.DataFrame(prob)
prob

,Survived
Sex,
female,0.779736
male,0.186869


In [59]:
#probabilty of died
prob["Died"]=1-prob["Survived"]

prob

,Survived,Died
Sex,,
female,0.779736,0.220264
male,0.186869,0.813131


In [60]:
#calculating weight of evidence
prob["WOE"]=np.log(prob["Survived"]/prob["Died"])


In [61]:
prob

,Survived,Died,WOE
Sex,,,
female,0.779736,0.220264,1.264127
male,0.186869,0.813131,-1.470486


In [62]:
#creating a function that calculate woe and apply on train and test

def WOE(train, test, var, target):
    
    prob = pd.DataFrame(train.groupby([var])[target].mean())
    
    prob["bad"]=1-prob[target]
    
    mapper = (np.log(prob[target]/prob["bad"])).to_dict()
    
    train[var] = train[var].map(mapper)
    test[var] = test[var].map(mapper)


In [63]:
cat_var = ["Sex","Cabin","Embarked_imputed"]

In [64]:
#applying the function
for var in cat_var:
    WOE(X_train,X_test,var, "Survived")

In [65]:
X_train.head()

,Sex,Cabin,Embarked_imputed,Survived
857,-1.470486,0.682676,0.322083,0
52,-1.470486,-0.730888,-0.594707,0
386,-1.470486,-0.730888,-0.594707,0
124,-1.470486,-0.730888,0.322083,0
578,1.264127,0.682676,-0.594707,1


In [66]:
X_test.head()

,Sex,Cabin,Embarked_imputed,Survived
495,-1.470486,0.682676,-0.594707,0
648,-1.470486,-0.730888,-0.594707,0
278,-1.470486,-0.730888,-0.594707,0
31,-1.470486,-0.730888,-0.594707,1
255,1.264127,-0.730888,-0.594707,0
